## Notebook 2PT step by step.

###### A partir de trayectorias de DM molecular generadas con LAMMPS, se utliza el metodo del 2PT para calcular propiedades termodinámicas.




La primer parte consiste en procesar las trayectorias para obtener las DoS.

El dump de lammps tiene el siguiente formato:


```
  dump 1 all custom #Steps Fname id mol x y z fx fy fz vx vy vz
  dump_modify 1 sort id 
```

Y las variables termodinámicas:


```
thermo_style custom step temp pe ke etotal vol press epair emol ebond eangle enthalpy density lx ly lz
```



En el mismo directorio donde se encuentre la notebook tiene que haber una carpeta PX.X que contiene el conjunto de trayectorias a presion X.X(archivos .trj y thermo_ XXX.log) . (Se puede crear y subir a mano con el menú de la izquierda o ejecutar el siguiente bloque)



In [ ]:
from google.colab import files
import os

t = 1.0
p = 1.0
#Select working folder
variable = 'P'              #Variable a barrer presión o temperatura
if variable == 'T':
    folder = variable+str(t)
elif variable == 'P':
    folder = variable+str(p)

path =  ('{}/'.format(folder))
os.makedirs(path, exist_ok=True)
os.chdir(folder)

data_to_load = files.upload()
os.chdir('/content/')

Con las trayectorias cargadas ya es posible calcular las DoS trn, rot y vib.

Ejecutar el siguiente bloque para cargar las funciones necesarias.

In [ ]:
import scipy as sp
import numpy as np
import linecache

 
class Trajectory:
    def __init__(self, filename, attr, skip):
        """ 
        filename         : path to the trajectory file with sorted and format = id mol x y z fx fy fz vx vy vz
        attr             : particles attributes to be loaded in memory = coordinates(0), forces(1) or velocities(2)  
        skip             : number of snapshots to be skipped between two configurations that are evaluated
                           (for example, if trajectory is 9000 steps long, and skip = 10, every tenth step
                           is evaluated, 900 steps in total; use skip = 1 to take every step of the MD)    
         """
         
        with open(filename, 'r') as f:
            data = [next(f) for x in range(10)]  #solo leo las primeras 10 lineas del archivo para  sacar natoms                  
        self.n_atoms = int(data[3].split()[0])
        f.close() 

        
        with open(filename,'r') as f:
            count = 0
            for line in f:
                count +=1
            self.n_steps_total = count // (self.n_atoms + 9)
        f.close()
        
        self.skip = skip
        self.n_steps = self.n_steps_total // self.skip 

        attributes = ('coordinates', 'forces', 'velocities')
        print('Are going to be loaded: particles {}'.format(attributes[attr]))
        print('Trajectory frames= ',self.n_steps_total)
        print('Frames to be loaded= ',self.n_steps) 
        

        self.coordinates = np.zeros((self.n_steps, self.n_atoms, 3))
        count = 0
        stop = self.n_steps
        for step in range(self.n_steps):
            print('Loading Frame:', step*skip,' ',end='\x1b[1K\r')
            coords = np.zeros((self.n_atoms, 3))
            i = step * self.skip * (self.n_atoms + 9)
            data = linecache.getlines(filename)[i + 9 : i + self.n_atoms + 9]

            for j, line in enumerate(data):
                try:
                    coords[j, :] = [float(value) for value in line.split()[2+3*attr:5+3*attr]]
                except:
                    if count == 0:
                        stop = step
                        print( 'file broken in step: ',step * self.skip)
                    count += 1
                    break 
            self.coordinates[step] = coords
            del data
        self.coordinates =  self.coordinates[:stop,:,:]    
        
        if attr == 0: 
            self.boxsize = np.zeros((self.n_steps, 3, 2))
            for step in range(self.n_steps): #array with box sizes
                box = np.zeros((3, 2))             
                i = step * self.skip * (self.n_atoms + 9)
                data = linecache.getlines(filename)[i + 5 : i + 8]
                for j, line in enumerate(data):
                    try:
                        box[j, :] = [float(value) for value in line.split()[0:]] 
                    except:
                        break
                self.boxsize[step] = box  
                del data
            self.boxsize = self.boxsize[:stop,:,:]
  

    def Return_DOS_trn(self,tstep,Dstep,temp,m,nb,fc):
        """This function return the translational (DOS). 
        1° Takes de power spectral density  of  molecule's  mass center velocities, 
        2° Sum it for x,y,z and whole system 
        3° weigh the sum with kb*T 
        Returns: freq [1/s] ; Dos_trn[tot,x,y,z] [s] 
        Needs:
        *self.coordinates: atoms velocities [a.u.]
        *tstep: simulation timestep [s]
        *Dstep: every few timestep each frame is recorded []
        *temp: temperature [K]
        *m: atoms mass (sorted array for molecule)[kg]
        *nb: atoms per molecule []
        *fc: conversion factor to have velocities  [m/s].   
        """
        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzman constant [J/K] 
        nm = self.n_atoms//nb                             #molecules number
        Ts = tstep*Dstep*self.skip                        #sampling period
        mi = np.sum(m)                                    #molecule mass
        T = temp                                          #simulation temperature
        n = self.n_steps*4                                #number of points to do fft (if n>steps vector it will be zero padded) []
        self.coordinates *= fc

        CMvs = np.zeros((self.n_steps,nm, 3))   #molecule mass center velocities
        for step in range(self.n_steps):
            data_frame = np.array(self.coordinates[step])
            for mol in np.arange(1,nm+1,1):
                beadvs = data_frame[(mol-1)*nb:(mol*nb),:]    #beads velocitys in each particle              
                CMvs[step,mol-1,:] = np.average(beadvs, axis=0, weights=m)   #molecule mass center velocity
        
        fft  = np.zeros((n//2 +1,4)) #Sigle sided fft (only freq > 0 )
        for mol in range(nm):
            CMv = CMvs[:,mol,:]
            for i in range(3):   #(xyz)
                ffti = ( Ts  / (self.n_steps-1))*(np.abs(np.fft.rfft(CMv[:,i], n=n))**2)*mi 
                fft[:, i+1] += ffti
                fft[:, 0] += ffti   
        
        freq = np.fft.rfftfreq(n, d=Ts)
        DOS = ((2.)/(kb*T))*fft
        return DOS, freq

    def Return_DOS_rot(self,rposis,tstep,Dstep,temp,m,nb,fc):
        """This function return the rotational (DOS).
        1° calculates the molecule mass translational velocity;
        2° Calculates the molecule angular velocity around principal axis;
        3° Takes de power spectral density  of  molecule  mass center angular velocities,
        4° Sum it for x,y,z and whole system
        5° weigh the sum with kb*T
        Returns: freq [1/s] ; Dos_rot[tot,a,b,c] [s] ; I [Kgm²]
        Needs:
        *self.coordinates: velocities [m/s]
        *rposi: distance [m]
        *tstep: simulation timestep [s]
        *Dstep: every few timestep each frame is recorded []
        *temp: temperature [K]
        *m: atom mass (sorted array for molecule)[kg]
        *nb: atoms per molecule []
        *fc: conversion factor to have velocities  in SI [m/s].

        References:
        (1)Application of the Eckart frame to soft matter: Rotation of star polymers under shear flow [2017]
        (2)Master thesis 2PT bernhartdt [2016]
        """
        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzman constant [J/K]
        nm = self.n_atoms//nb                             #molecules number
        Ts = tstep*Dstep*self.skip                        #sampling period
        T = temp                                          #simulation temperature
        n = self.n_steps*4                                #number of points to do fft (if n>steps vector it will be zero padded) []
        self.coordinates *= fc


        CMvs = sp.zeros((self.n_steps,nm, 3))   #molecule mass center velocities
        for step in range(self.n_steps):
            data_frame = np.array(self.coordinates[step])
            for mol in np.arange(1,nm+1,1):
                beadvs = data_frame[(mol-1)*nb:(mol*nb),:]    #beads velocities in each molecule
                CMvs[step,mol-1,:] = np.average(beadvs, axis=0, weights=m)   #molecule mass center velocity



        CMws = np.zeros((self.n_steps,nm, 3))   #molecule angular velocity
        Imom = np.zeros((self.n_steps,nm, 3))
        for mol in sp.arange(1,nm+1,1):
            for step in range(self.n_steps):

                data_rposi = rposis[step,(mol-1)*nb:(mol*nb),:]  #See eq 2 and 3 in: (1)
                data_vicm = self.coordinates[step,(mol-1)*nb:(mol*nb),:] - CMvs[step,mol-1,:]

                L = np.sum([np.cross(data_rposi[i,:],data_vicm[i,:]*m[i]) for i in range(nb) ],axis=0) #angular momentum
                J = np.sum( [ m[i]*(np.dot(data_rposi[i,:],data_rposi[i,:])*np.identity(3) \
                            - np.tensordot(data_rposi[i,:],data_rposi[i,:],axes=0)) for i in range(nb) ],axis=0) #inertia matrix

                eival, eivec = np.linalg.eig(J) #Principal intertia  moments and axis
                idx = eival.argsort()[::-1] #highest to lowest
                eival = eival[idx]
                eivec = eivec[:,idx]
                Imom[step,mol-1,:] = eival
                #lines to avoid eigvect +1 o -1 directions
                if step == 0:
                    for i in range(3):
                        CMws[step,mol-1,i] = (1./eival[i])*np.dot(L,eivec[:,i])*np.sqrt(eival[i]) #See eq. 3.3 Master thesis 2PT bernhartdt
                    eivecprev = eivec
                else:
                    for i in range(3):
                        test = np.dot(eivec[:,i],eivecprev[:,i])
                        if test < 0.:
                            eivec[:,i] *= -1. #if eivec point in inverse direction from previous step
                        CMws[step,mol-1,i] = (1./eival[i])*np.dot(L,eivec[:,i])*np.sqrt(eival[i]) #srqt(I_jl)*w_jl
                    eivecprev = eivec

        fft  = np.zeros((n//2 + 1, 4)) #Sigle sided fft (only freq > 0 )
        for mol in range(nm):
            CMw = CMws[:,mol,:]
            for i in range(3):   #(xyz)
                ffti = ( Ts  / (self.n_steps-1))*(np.abs(np.fft.rfft(CMw[:,i], n=n))**2)
                fft[:, i+1] += ffti
                fft[:, 0] += ffti

        I = np.mean(Imom[:,:,:],axis=(0,1))
        freq = np.fft.rfftfreq(n, d=Ts)
        DOS = ((2.)/(kb*T))*fft
        return DOS, freq, I

    def Return_DOS_vib(self,rposis,tstep,Dstep,temp,m,nb,fc):
        """This function return the vibrational (DOS).
        1° calculates the molecule mass translational velocity;
        2° Calculates the molecule angular velocity around principal axis;
        3° Calculates atoms vibrational velocities.
        4° Takes de power spectral density  of  atoms  mass center vibrational velocities,
        5° Sum it for x,y,z and whole system
        6° weigh the sum with kb*T
        Returns: freq [1/s] ; Dos_vib[tot,x,y,z] [s]
        Needs:
        *self.coordinates: velocities [a.u.]
        *rposi: distance [m]
        *tstep: simulation timestep [s]
        *Dstep: every few timestep each frame is recorded []
        *temp: temperature [K]
        *m: atom mass (sorted array for molecule)[kg]
        *nb: atoms per molecule []
        *fc: conversion factor to have velocities  in SI [m/s].

        References:
        (1)Application of the Eckart frame to soft matter: Rotation of star polymers under shear flow [2017]
        (2)Master thesis 2PT bernhartdt [2016]
        """
        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzman constant [J/K]
        nm = self.n_atoms//nb                             #molecules number
        Ts = tstep*float(Dstep)*float(self.skip)                        #sampling period
        T = temp                                          #simulation temperature
        n = self.n_steps*4                                #number of points to do fft (if n>steps vector it will be zero padded) []
        self.coordinates *= fc


        CMvs = np.zeros((self.n_steps,nm, 3))   #molecule mass center velocities
        for step in range(self.n_steps):
            data_frame = np.array(self.coordinates[step])
            for mol in np.arange(1,nm+1,1):
                beadvs = data_frame[(mol-1)*nb:(mol*nb),:]    #beads velocities in each molecule
                CMvs[step,mol-1,:] = np.average(beadvs, axis=0, weights=m)   #molecule mass center velocity



        CMws  = np.zeros((self.n_steps,nm, 3))   #molecule angular velocity
        vibvs = np.zeros((self.n_steps,self.n_atoms, 3)) # atoms vibrational velocities
        for mol in np.arange(1,nm+1,1):
            for step in range(self.n_steps):

                data_rposi = rposis[step,(mol-1)*nb:(mol*nb),:]  #See eq 2 and 3 in: (1)
                data_vicm = self.coordinates[step,(mol-1)*nb:(mol*nb),:] - CMvs[step,mol-1,:]

                L = np.sum([np.cross(data_rposi[i,:],data_vicm[i,:]*m[i]) for i in range(nb) ],axis=0) #angular momentum
                J = np.sum( [ m[i]*(np.dot(data_rposi[i,:],data_rposi[i,:])*np.identity(3) \
                            - np.tensordot(data_rposi[i,:],data_rposi[i,:],axes=0)) for i in range(nb) ],axis=0) #inertia matrix

                CMws[step,mol-1,:] = np.dot(np.linalg.inv(J),L)
                vibvs[step,(mol-1)*nb:(mol*nb),:] = (np.array(self.coordinates[step,(mol-1)*nb:(mol*nb),:]) - \
                                                     (CMvs[step,mol-1,:] + np.cross(CMws[step,mol-1,:], rposis[step,(mol-1)*nb:(mol*nb),:])))



        fft  = np.zeros((n//2 +1, 4)) #Sigle sided fft (only freq > 0 )
        for atom in range(self.n_atoms):
            vibv = vibvs[:,atom,:]
            for i in range(3):   #(xyz)
                ffti = ( Ts  / (self.n_steps-1))*(np.abs(np.fft.rfft(vibv[:,i], n=n))**2)*m[i]
                fft[:,i+1] += ffti
                fft[:,0] += ffti

        freq = np.fft.rfftfreq(n, d=Ts)
        DOS = ((2.)/(kb*T))*fft
        return DOS, freq

    def compute_rposi(self,m,nb,fc):
        """This function calculates atom vector position to
        molecule mass center (r_i - r_cm).
        Needs:
        *self.coordinates: atoms positions [a.u.]
        *m: atom mass (sorted array for molecule)[kg]
        *nb: atoms per molecule []
        *fc: conversion factor to have positions in [m].
        """
        from time import sleep
        nm = self.n_atoms//nb #molecules number
        self.coordinates *= fc
        self.boxsize *= fc

        rposis = np.zeros((self.n_steps, self.n_atoms, 3))
        for step in range(self.n_steps):
            data_box    = sp.array(self.boxsize[step])#Step box size
            A = (data_box[0,1] - data_box[0,0])*0.5
            B = (data_box[1,1] - data_box[1,0])*0.5
            C = (data_box[2,1] - data_box[2,0])*0.5
            box = np.array([A,B,C])

            data_beads = sp.array(self.coordinates[step,:,:])
            for mol in sp.arange(1,nm+1,1):
                molecule = data_beads[(mol-1)*nb:(mol*nb),:]

                moleculeu = np.zeros(molecule.shape) #molecule with unwrapped cordinates
                moleculeu[0,:] = molecule[0,:]
                for i in range(nb-1):
                    dist = (molecule[i+1]-moleculeu[i])
                    for xyz in (0,1,2):
                        if dist[xyz]>box[xyz]:
                            moleculeu[i+1,xyz] = molecule[i+1,xyz] - box[xyz]*2.
                        elif dist[xyz]<=(-box[xyz]):
                            moleculeu[i+1,xyz] = molecule[i+1,xyz] + box[xyz]*2.
                        else:
                            moleculeu[i+1,xyz] = molecule[i+1,xyz]
                CM = np.average(moleculeu, axis=0, weights=m)
                rposis[step,(mol-1)*nb:(mol*nb),:] = moleculeu[:,:] - CM[:]
        return rposis

    def Return_DOS_partition_trn(self,DOSt,freq,tstep,Dstep,temp,m,nb,rho):
        """This function return the translational Dos_g_trn gas 
        and Dos_s_trn solid, Delta and fluidicity f. 
        Returns: c*Dos_trn [m] c*Dos_g_trn [m] ; c*Dos_s_trn [m] ; freq/c [1/m] ;
        f [] ; Delta []  
        Needs:
        *DOSt: Total density of states [s]
        *freq: frequencies [1/s] 
        *tstep: simulation timestep [s]
        *Dstep: every few timestep each frame is recorded []
        *temp: temperature [K]
        *m: atoms mass (sorted array for molecule)[kg]
        *nb: atoms per molecule []
        *rho: density in [kg/m3]
        """
        import scipy.optimize as opt
        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzman constant [J/K]
        c = 299792458.                                    #light speed in [m/s]
        nm = self.n_atoms//nb                             #particles number
        Ts = tstep*Dstep*self.skip                        #sampling period
        mi = np.sum(m)                                    #particle mass
        T = temp                                          #simulation temperature
 
        cDOSt = c*DOSt
        vfreq = freq/c
        #fluidicity f
        cdos0 = cDOSt[0] #gas-like difussive component
        Delta = (2*cdos0/(9*nm))*np.sqrt((np.pi*kb*T)/mi)*((rho)**(1/3))*((6/np.pi)**(2/3)) 
 
        def y(f,Delta):
            return 2*(Delta**(-9/2))*(f**(15/2)) - 6*(Delta**(-3))*(f**5) - \
            (Delta**(-3/2))*(f**(7/2)) + 6*(Delta**(-3/2))*(f**(5/2)) + \
            2*f - 2

        f = opt.root_scalar(y, args=(Delta,), method='toms748', bracket=(0.,1.), rtol=1.0e-6, maxiter=int(1e6)).root #Busqueda de la raiz entre 0 y 1.

        cDOSg =  cdos0 / (1 + ((np.pi*cdos0*vfreq) /(6*f*nm))**2)       #DOS gas-like component
        cDOSs = cDOSt - cDOSg                  #DOS solid-like component

        return cDOSt, cDOSg, cDOSs, vfreq, f, Delta

    def Return_thermoproperties_trn(self,cDOSg,cDOSs,vfreq,T,m,f,Delta,V,nb,stat):
        """Function that return Internal Energy, Entropy and Helmoltz Free Energy
        only for rotational component.
        Returns: E/particle [J]; S/particle [J/K] ; A/particle [J]
        Needs:
        * c*Dos_g_trn: gas-like DOS [m] 
        * c*Dos_s_trn solid-like DOS[m] 
        * freq/c: wavenumber [1/m] 
        * T: temperature [K]
        * m: particle mass [kg]
        * f: fluidicity parameter []
        * Delta: normalized diffusivity []
        * V: Box volume [m3]
        * nb: beads per particle []
        * stat: quantum or classic solid-like weighting function ['q' or 'c']
        * Emd: Molecular dynamics total(internal) energy [J].
        """
        import scipy.integrate as integrate

        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzmann's constant [J/K]
        h = 6.62607015e-34                                #Planck's constant  [J*s]
        c = 299792458.                                    #light speed in [m/s]
        nm = self.n_atoms//nb                             #particles number
        mi = np.sum(m)                                    #particle mass

        #Energy
        B = 1/(kb*T)
        Bhv = B*h*(vfreq*c)
        if stat == 'c':
            Wes = 1                             #classical weight
        elif stat == 'q':
            Wes = Bhv*0.5 + Bhv / (np.exp(Bhv)-1)#Quantum weight
            Wes[0] = 1. 
        Es = integrate.simps(cDOSs*Wes,x=vfreq)

        Weg = 0.5
        Eg =  integrate.simps(cDOSg*Weg,x=vfreq)

        E = (Es+Eg)/B
        E /= nm


        #Entropy S
        if stat == 'c':   
            Wss = 1 - np.log(Bhv)                            #Classic weighting
            Wss[0] = 1 - np.log(1.e-323) 
        elif stat == 'q':
            Wss = Bhv / (np.exp(Bhv)-1) - np.log(1 - np.exp(-Bhv)) #quatum weighting
            Wss[0] = 0. 
        Ss =  integrate.simps(cDOSs*Wss,x=vfreq)

        y = (f**(5./2.)) / (Delta**(3./2.))
        z = (1 + y + y**2 - y**3 ) / ((1 - y)**3)
        Wsg =(1./3.)*((5./2.) + np.log(((2.*np.pi*mi*kb*T)/ h**2)**(3./2.) * ((V*z)/(f*nm))) + y*(3*y-4)/(1-y)**2 )
        Sg =  integrate.simps(cDOSg*Wsg,x=vfreq)

        S =kb*(Ss+Sg)
        S /= nm


        #Helmoltz Free Energy
        if stat == 'c':                              #Classic weighting
            Was = np.log(Bhv)
            Was[0] = np.log(1.e-323)
        elif stat == 'q':
            Was = np.log((1-np.exp(-Bhv))/(np.exp(-Bhv/2.))) #quatum weighting
            Was[0] = np.log(1.e-323)
        As = integrate.simps(cDOSs*Was,x=vfreq)

        Wag = Weg - Wsg
        Ag =  integrate.simps(cDOSg*Wag,x=vfreq)

        A =  (As+Ag)/B
        A /= nm


        return E, S, A

    def Return_DOS_partition_rot(self,DOSt,freq,tstep,Dstep,temp,m,nb,rho):
        """This function return the rotational Dos_g_trn gas 
        and Dos_s_trn solid, Delta and fluidicity f. 
        Returns: c*Dos_rot [m] c*Dos_g_rot [m] ; c*Dos_s_rot [m] ; freq/c [1/m] ;
        f [] ; Delta []  
        Needs:
        *DOSt: Total density of states [s]
        *freq: frequencies [1/s]
        *tstep: simulation timestep [s]
        *Dstep: every few timestep each frame is recorded [] 
        *temp: temperature [K]
        *m: atoms mass (sorted array for molecule)[kg]
        *nb: atoms per molecule []
        *rho: density in [kg/m3]
        """
        import scipy.optimize as opt
        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzman constant [J/K]
        c = 299792458.                                    #light speed in [m/s]
        nm = self.n_atoms//nb                             #particles number
        Ts = tstep*Dstep*self.skip                        #sampling period
        mi = np.sum(m)                                    #particle mass
        T = temp                                          #simulation temperature
 
        cDOSt = c*DOSt
        vfreq = freq/c
        #fluidicity f
        cdos0 = cDOSt[0] #gas-like difussive component
        Delta = (2*cdos0/(9*nm))*np.sqrt((np.pi*kb*T)/mi)*((rho)**(1/3))*((6/np.pi)**(2/3)) 
 
        def y(f,Delta):
            return 2*(Delta**(-9/2))*(f**(15/2)) - 6*(Delta**(-3))*(f**5) - \
            (Delta**(-3/2))*(f**(7/2)) + 6*(Delta**(-3/2))*(f**(5/2)) + \
            2*f - 2

        f = opt.root_scalar(y, args=(Delta,), method='toms748', bracket=(0.,1.), rtol=1.0e-6, maxiter=int(1e6)).root #Busqueda de la raiz entre 0 y 1.

        cDOSg =  cdos0 / (1 + ((np.pi*cdos0*vfreq) /(6*f*nm))**2)       #DOS gas-like component
        cDOSs = cDOSt - cDOSg                  #DOS solid-like component

        return cDOSt, cDOSg, cDOSs, vfreq, f, Delta

    def Return_thermoproperties_rot(self,cDOSg,cDOSs,vfreq,T,I,f,Delta,V,nb,stat,sigma):
        """Function that return Internal Energy, Entropy and Helmoltz Free Energy
        only for rotational component.
        Returns: E/particle [J]; S/particle [J/K] ; A/particle [J]
        Needs:
        * c*Dos_g_rot: gas-like DOS [m] 
        * c*Dos_s_rot solid-like DOS[m] 
        * freq/c: wavenumber [1/m] 
        * T: temperature [K]
        * I: Principal Moments Of Inertia (sorted array for molecule)[Kgm²]
        * f: fluidicity parameter []
        * Delta: normalized diffusivity []
        * V: Box volume [m3]
        * nb: beads per particle []
        * stat: quantum or classic solid-like weighting function ['q' or 'c']
        * sigma: Rotational symmetry []
        """
        import scipy.integrate as integrate

        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzmann's constant [J/K]
        h = 6.62607015e-34                                #Planck's constant  [J*s]
        c = 299792458.                                    #light speed in [m/s]
        nm = self.n_atoms//nb                             #particles number

        #Energy
        B = 1/(kb*T)
        Bhv = B*h*(vfreq*c)
        if stat == 'c':
            Wes = 1                             #classical weight
        elif stat == 'q':
            Wes = Bhv*0.5 + Bhv / (np.exp(Bhv)-1)#Quantum weight
            Wes[0] = 1. 
        Es = integrate.simps(cDOSs*Wes,x=vfreq)

        Weg = 0.5
        Eg =  integrate.simps(cDOSg*Weg,x=vfreq)

        E =  (Es+Eg)/B
        E /= nm

        #Entropy S
        if stat == 'c':   
            Wss = 1 - np.log(Bhv)                            #Classic weighting
            Wss[0] = 1 - np.log(1.e-323) 
        elif stat == 'q':
            Wss = Bhv / (np.exp(Bhv)-1) - np.log(1 - np.exp(-Bhv)) #quatum weighting
            Wss[0] = 0. 
        Ss =  integrate.simps(cDOSs*Wss,x=vfreq)

        y = (f**(5./2.)) / (Delta**(3./2.))
        z = (1 + y + y**2 - y**3 ) / ((1 - y)**3)
        Wsg =(1./3.)*((5./2.) + np.log(((2.*np.pi*m*kb*T)/ h**2)**(3./2.) * ((V*z)/(f*nm))) + y*(3*y-4)/(1-y)**2 )

        tit= np.array([ (h**2) / (8.*np.pi*I[i]*kb) for i in range(3)])
        Wsg = (1./3.)*np.log( ( ((np.pi**0.5)*np.exp(3/2)) / sigma ) *(T**3 / (tit[0]*tit[1]*tit[2])**0.5))
        Sg =  integrate.simps(cDOSg*Wsg,x=vfreq)

        S =kb*(Ss+Sg)
        S /= nm

        #Helmoltz Free Energy
        if stat == 'c':                              #Classic weighting
            Was = np.log(Bhv)
            Was[0] = np.log(1.e-323)
        elif stat == 'q':
            Was = np.log((1-np.exp(-Bhv))/(np.exp(-Bhv/2.))) #quatum weighting
            Was[0] = np.log(1.e-323)
        As = integrate.simps(cDOSs*Was,x=vfreq)

        Wag = Weg - Wsg
        Ag =  integrate.simps(cDOSg*Wag,x=vfreq)

        A =  (As+Ag)/B
        A /= nm

        return E, S, A

    def Return_DOS_partition_vib(self,DOSt,freq):
        """This function return the vibrational Dos_s_vib solid, 
        fluidicity f = 0 . 
        Returns: c*Dos_vib [m]; c*Dos_s_vib [m] ; freq/c [1/m] ;  
        Needs:
        *DOSt: Total density of states [s]
        *freq: frequencies [1/s] 
        """
        import scipy.optimize as opt
        c = 299792458.                                    #light speed in [m/s]
 
        cDOSt = c*DOSt
        vfreq = freq/c       
        cDOSs = cDOSt                   #DOS solid-like component

        return cDOSt, cDOSs, vfreq
  
    def Return_thermoproperties_vib(self,cDOSs,vfreq,T,nb,stat):
        """Function that return Internal Energy, Entropy and Helmoltz Free Energy
        only for vibrational component.
        Returns: E/particle [J]; S/particle [J/K] ; A/particle [J]
        Needs:
        * c*Dos_s_vib solid-like DOS[m] 
        * freq/c: wavenumber [1/m] 
        * T: temperature [K]
        * nb: beads per particle []
        * stat: quantum or classic solid-like weighting function ['q' or 'c']
        """
        import scipy.integrate as integrate

        kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzmann's constant [J/K]
        h = 6.62607015e-34                                #Planck's constant  [J*s]
        c = 299792458.                                    #light speed in [m/s]
        nm = self.n_atoms//nb                             #particles number


        #Energy
        B = 1/(kb*T)
        Bhv = B*h*(vfreq*c)
        if stat == 'c':
            Wes = 1                             #classical weight
        elif stat == 'q':
            Wes = Bhv*0.5 + Bhv / (np.exp(Bhv)-1)#Quantum weight
            Wes[0] = 1. 
        Es = integrate.simps(cDOSs*Wes,x=vfreq)

        E = (Es)/B
        E /= nm

        #Entropy S
        if stat == 'c':   
            Wss = 1 - np.log(Bhv)                            #Classic weighting
            Wss[0] = 1 - np.log(1.e-323) 
        elif stat == 'q':
            Wss = Bhv / (np.exp(Bhv)-1) - np.log(1 - np.exp(-Bhv)) #quatum weighting
            Wss[0] = 0. 
        Ss =  integrate.simps(cDOSs*Wss,x=vfreq)

        S =kb*(Ss)
        S /= nm

        #Helmoltz Free Energy
        if stat == 'c':                              #Classic weighting
            Was = np.log(Bhv)
            Was[0] = np.log(1.e-323)
        elif stat == 'q':
            Was = np.log((1-np.exp(-Bhv))/(np.exp(-Bhv/2.))) #quatum weighting
            Was[0] = np.log(1.e-323)
        As = integrate.simps(cDOSs*Was,x=vfreq)

        A =  (As)/B
        A /= nm

        return E, S, A

Proceder al calculo definiendo los paramentros de la simulacion.

El parametro "nruns" indica el número de corridas a procesar .

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os


#CH2 parameters (From McCoy1995)
Sigma = 3.56 * 1.e-10                             #Sigma in [m]
kb = 0.0019872067*(4186.8 / 1.)/6.02214076e23     #Boltzmann's constant [J/K]
Epsilon = 70.47 * kb                              #Epsilon in [J]
mch2 = 14.026 *(1. / 1000.)/6.02214076e23        #CH2 mass in [kg]

#Simulation type and time
ens = 'NVT'
Dstep = 16              #frames dump
tstep = 0.001*np.sqrt(mch2 / Epsilon)*Sigma  #time step in second
skip = 1                #If don't want load all frames
nb = 13                  #particles per molecule
fc = (Sigma / (np.sqrt(mch2 / Epsilon)*Sigma))    #conversion factor lj units to [m/s]
fc2 = Sigma                                      #conversion factor lj to SI in [m]

T   =  t * (Epsilon/kb)
m = np.array([mch2, mch2,mch2, mch2, mch2, mch2\
              ,mch2, mch2, mch2, mch2,mch2, mch2, mch2])  #Atoms mass in kg


path =  ('{}/PostProc/'.format(folder))
os.makedirs(path, exist_ok=True)

nruns = 20

#DoS trn
j = 1
for i in np.arange(j,j+nruns):
    #INPUT 
    file =  ('{}/lc2PT_{}.DS{}.t{:g}p{}.trj'
            .format(folder,i,Dstep,t,p))

    print('Procesando:',file)
    trj = Trajectory(file, 2,skip)
    DOSt, freq = trj.Return_DOS_trn(tstep,Dstep,T,m,nb,fc)

    #OUTPUT
    dir =  ('{}/PostProc/Dostrn{}_{}.DS{}t*{}p*{}.txt'
            .format(folder,ens,i,Dstep,t,p))

    out_file = open(dir, "w")
    np.savetxt(out_file, (np.vstack((freq[:], DOSt[:,0], DOSt[:,1], DOSt[:,2], DOSt[:,3])).T),\
                fmt='%E',header='freq [1/s], DOSt[s], DOSx[s], DOSy[s], DOSz[s]')
    out_file.close()
    print('Procesado:', file)

#DoS rot
j = 1
for i in np.arange(j,j+nruns):
    #INPUT
    file =  ('{}/lc2PT_{}.DS{}.t{:g}p{}.trj'
            .format(folder,i,Dstep,t,p))

    print('Procesando:',file)
    coords = Trajectory(file, 0,skip)
    rposi = coords.compute_rposi(m,nb,fc2)
    del coords
    vels = Trajectory(file, 2,skip)
    DOSt, freq, I = vels.Return_DOS_rot(rposi,tstep,Dstep,T,m,nb,fc)

    #OUTPUT
    dir =  ('{}/PostProc/Dosrot{}_{}.DS{}t*{}p*{}.txt'
            .format(folder,ens,i,Dstep,t,p))
    header='freq [1/s], DOSt[s], DOSa[s], DOSb[s], DOSc[s] / I = {}'.format(I)
    out_file = open(dir, "w")
    np.savetxt(out_file, (np.vstack((freq[:], DOSt[:,0], DOSt[:,1], DOSt[:,2], DOSt[:,3])).T),\
                fmt='%E',header=header)
    out_file.close()
    print('Procesado:', file)
    del vels

#DoS vib
j = 1
for i in np.arange(j,j+nruns):
    #INPUT

    file =  ('{}/lc2PT_{}.DS{}.t{:g}p{}.trj'
            .format(folder,i,Dstep,t,p))

    print('Procesando:',file)
    coords = Trajectory(file, 0,skip)
    rposi = coords.compute_rposi(m,nb,fc2)
    del coords
    vels = Trajectory(file, 2,skip)
    DOSt, freq = vels.Return_DOS_vib(rposi,tstep,Dstep,T,m,nb,fc)

    #OUTPUT
    dir =  ('{}/PostProc/Dosvib{}_{}.DS{}t*{}p*{}.txt'
            .format(folder,ens,i,Dstep,t,p))

    out_file = open(dir, "w")
    np.savetxt(out_file, (np.vstack((freq[:], DOSt[:,0], DOSt[:,1], DOSt[:,2], DOSt[:,3])).T),\
                fmt='%E',header='freq [1/s], DOSt[s], DOSx[s], DOSy[s], DOSz[s]')
    out_file.close()
    print('Procesado:', file)
    del vels

Una vez que se tiene la DoS para cada trayectoria es momento de separar la componente gaseosa y la solida para luego calcular Energía interna, entropía y energía libre para cada grado libertad.

Cada uno de los siguientes bloques genera un archivo que tiene "nruns" filas indicando f [], E/p [J], S/p [J/K], A/p [J].


Traslacional

In [ ]:
#Processing Params
dos = 'trn'
stat = 'q'


thermo  = np.zeros(( nruns,4))
file =  ('{}/lc2PT_{}.DS{}.t{:g}p{}.trj'
                .format(folder,1,Dstep,t,p))
trj = Trajectory(file, 2,skip=550)  #Solo para poder usar metodos del Class Trajectory

for i in range(1,nruns+1):
    V  = np.genfromtxt(('{}/thermo_nvt_{}_t{:g}p{}.log'
                        .format(folder,i,t,p)) , skip_header=45,
                        invalid_raise = False)[0,5]*(Sigma**3) #vol in [m3] 
    rho =  (59904*mch2)/V         #Density in [Kg/m3]

    try:
        dir =  ('P{}/PostProc/Dos{}{}_{}.DS{}t*{}p*{}.txt'
                .format(p,dos,ens,i,Dstep,t,p))
        DOS_data = np.genfromtxt(dir, skip_header=0,invalid_raise = False)[:,:]
    except:
        dir =  ('P{}/PostProc/Dos{}{}_{}.DS{}t_{}p_{}.txt'
                .format(p,dos,ens,i,Dstep,t,p))      
        DOS_data = np.genfromtxt(dir, skip_header=0,invalid_raise = False)[:,:]      

    DOSt = np.zeros((nruns,4))
    freq, DOSt= DOS_data[:,0], DOS_data[:,1:]
    cDOSt, cDOSg, cDOSs, vfreq, f, Delta= trj.Return_DOS_partition_trn(DOSt[:,0],freq,tstep,Dstep,T,m,nb,rho)
    Ei, Si, Ai = trj.Return_thermoproperties_trn(cDOSg,cDOSs,vfreq,T,m,f,Delta,V,nb,stat)
    thermo[i-1,:] = [f,Ei,Si,Ai]   


#OUTPUT
import os
path =  ('{}/PostProc/DOS_thermo/'.format(folder))
os.makedirs(path, exist_ok=True)
dir_o =  ('{}/PostProc/DOS_thermo/DOS_thermo_trn{}.DS{}t*{}p*{}.txt'
        .format(folder,ens,Dstep,t,p))
out_file = open(dir_o, "w")
np.savetxt(out_file, thermo[:,:],\
            fmt='%E',header='f [], E/p [J], S/p [J/K], A/p [J]')
out_file.close()
print('Procesado:', dir_o)

Rotacional

In [ ]:
#Processing Params
dos = 'rot'
stat = 'q'


thermo  = np.zeros(( nruns,4))
file =  ('{}/lc2PT_{}.DS{}.t{:g}p{}.trj'
        .format(folder,1,Dstep,t,p))
trj = Trajectory(file, 2,skip=550) #Solo para poder usar metodos del Class Trajectory

for i in range(1,nruns+1):
    V  = np.genfromtxt(('{}/thermo_nvt_{}_t{:g}p{}.log'
                        .format(folder,i,t,p)) , skip_header=45,
                        invalid_raise = False)[0,5]*(Sigma**3) #vol in [m3] 
    rho =  (59904*mch2)/V         #Density in [Kg/m3]

    try:
        dir =  ('{}/PostProc/Dos{}{}_{}.DS{}t*{}p*{}.txt'
                .format(folder,dos,ens,i,Dstep,t,p))
        DOS_data = np.genfromtxt(dir, skip_header=0,invalid_raise = False)[:,:]
    except:
        dir =  ('{}/PostProc/Dos{}{}_{}.DS{}t_{}p_{}.txt'
                .format(folder,dos,ens,i,Dstep,t,p))
        DOS_data = np.genfromtxt(dir, skip_header=0,invalid_raise = False)[:,:]
    DOSt = np.zeros((nruns,4))
    freq, DOSt= DOS_data[:,0], DOS_data[:,1:]

    with open(dir) as f:
        string = f.readline().replace(']','').replace('[','').split()[-3:]
        I = [float(string[0]), float(string[1]), float(string[2])] #Average Inertia mom

    cDOSt, cDOSg, cDOSs, vfreq, f, Delta= trj.Return_DOS_partition_rot(DOSt[:,0],freq,tstep,Dstep,T,m,nb,rho)
    Ei, Si, Ai = trj.Return_thermoproperties_rot(cDOSg,cDOSs,vfreq,T,I,f,Delta,V,nb,stat,sigma=2.)
    thermo[i-1,:] = [f,Ei,Si,Ai]   


#OUTPUT
import os
path =  ('{}/PostProc/DOS_thermo/'.format(folder))
os.makedirs(path, exist_ok=True)
dir_o =  ('{}/PostProc/DOS_thermo/DOS_thermo_{}{}.DS{}t*{}p*{}.txt'
        .format(folder,dos,ens,Dstep,t,p))
out_file = open(dir_o, "w")
np.savetxt(out_file, thermo[:,:],\
            fmt='%E',header='f [], E/p [J], S/p [J/K], A/p [J]')
out_file.close()
print('Procesado:', dir_o)

Vibracional

In [ ]:
#Processing Params
dos = 'vib'
stat = 'q'


thermo  = np.zeros(( nruns,4))
file =  ('{}/lc2PT_{}.DS{}.t{:g}p{}.trj'
        .format(folder,20,Dstep,t,p))
trj = Trajectory(file, 2,skip=600)

for i in range(1,nruns+1):
    try:
        dir =  ('{}/PostProc/Dos{}{}_{}.DS{}t*{}p*{}.txt'
                .format(folder,dos,ens,i,Dstep,t,p))
        DOS_data = np.genfromtxt(dir, skip_header=0,invalid_raise = False)[:,:]
    except:
        dir =  ('{}/PostProc/Dos{}{}_{}.DS{}t_{}p_{}.txt'
                .format(folder,dos,ens,i,Dstep,t,p))
        DOS_data = np.genfromtxt(dir, skip_header=0,invalid_raise = False)[:,:]    
                
    DOSt = np.zeros((nruns,4))
    freq, DOSt= DOS_data[:,0], DOS_data[:,1:]

    cDOSt, cDOSs, vfreq= trj.Return_DOS_partition_vib(DOSt[:,0],freq) 
    Ei, Si, Ai = trj.Return_thermoproperties_vib(cDOSs,vfreq,T,nb,stat) 
    f = 0.
    thermo[i-1,:] = [f,Ei,Si,Ai]   


#OUTPUT
import os
path =  ('{}/PostProc/DOS_thermo/'.format(folder))
os.makedirs(path, exist_ok=True)
dir_o =  ('{}/PostProc/DOS_thermo/DOS_thermo_{}{}.DS{}t*{}p*{}.txt'
        .format(folder,dos,ens,Dstep,t,p))
out_file = open(dir_o, "w")
np.savetxt(out_file, thermo[:,:],\
            fmt='%E',header='f [], E/p [J], S/p [J/K], A/p [J]')
out_file.close()
print('Procesado:', dir_o)